In [ ]:
import os
from speechbrain.pretrained import SepformerSeparation as separator
import soundfile
import torchaudio
from pydub import AudioSegment
from IPython.display import Audio 
from IPython.core.display import display
import os
from scipy.io import wavfile
import shutil
import noisereduce as nr
from tqdm import tqdm

C:\Users\Arian\AppData\Local\Temp\ipykernel_24756\2507175996.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [5]:
# sota_model= separator.from_hparams(source="speechbrain/sepformer-wsj02mix", savedir=f'pretrained_models/sepformer-wsj02mix',run_opts={"device":"cuda"})

In [4]:
# model_names = ["no_noise_speedperturb","w_noise_speedperturb","w_noise_wavedrop", "standard_model"]
model_names = ["standard_model","w_noise_wavedrop"]
models = {}
for model_name in model_names:
   models[model_name]= separator.from_hparams(source="speechbrain/sepformer-libri2mix", savedir=f'models/{model_name}',run_opts={"device":"cuda"})


In [16]:
input_path = 'mix'
output_path = 'audio_results'

file_names = os.listdir('mix')
for i,file in tqdm(enumerate(file_names), total=len(file_names)):
   for model_name in model_names:
      mix_path = "mix"
      mix_audio_path = file
      if model_name !=model_names[0]:
         mix_path = "synthetic_mix"
         mix_audio_path = f"item{i}_mix.wav"
      # print(f'{mix_path}/{mix_audio_path}')
      est_sources = models[model_name].separate_file(path=f'{mix_path}/{mix_audio_path}') 
      item_name = f'item{i}'
      # shutil.copy2(f'mix/{item_name}_mix.wav',f'{mix_path}/{mix_audio_path}',) 
      # item_name = file.split("_")[0]
      # display(Audio(f"{mix_path}/{file}"))
      audio1_filename = f"{output_path}/{item_name}_source1hat.wav"
      audio2_filename = f"{output_path}/{item_name}_source2hat.wav"

      torchaudio.save(audio1_filename, est_sources[:, :, 0].detach().cpu(), 8000)
      torchaudio.save(audio2_filename, est_sources[:, :, 1].detach().cpu(), 8000)
      
     
      rate1, audio1 = wavfile.read(audio1_filename)
      rate2, audio2 = wavfile.read(audio2_filename)
      audio1 = nr.reduce_noise(y=audio1, sr=8000)
      audio2 = nr.reduce_noise(y=audio2, sr=8000)
      
      wavfile.write(audio1_filename,8000, audio1)
      wavfile.write(audio2_filename,8000, audio2)
      
      audio1 = AudioSegment.from_file(audio1_filename)
      audio1 = audio1+6
      audio2 = AudioSegment.from_file(audio2_filename)
      audio2 = audio2+6
      
      mixed_audio = audio1.overlay(audio2)
      
      # mixed_audio.export(f"synthetic_mix/{model_name}_{item_name}_mix.wav",format="wav")
      
      mixed_audio.export(f"synthetic_mix\{item_name}_mix.wav",format="wav")

  0%|          | 0/3000 [00:00<?, ?it/s]C:\Users\Arian\AppData\Local\Temp\ipykernel_24756\1578545387.py:25: WavFileWarning: Chunk (non-data) not understood, skipping it.
  rate1, audio1 = wavfile.read(audio1_filename)
C:\Users\Arian\AppData\Local\Temp\ipykernel_24756\1578545387.py:26: WavFileWarning: Chunk (non-data) not understood, skipping it.
  rate2, audio2 = wavfile.read(audio2_filename)
  5%|▍         | 145/3000 [07:02<2:18:45,  2.92s/it]


KeyboardInterrupt: 

In [ ]:
# model = separator.from_hparams(source="speechbrain/sepformer-libri2mix", savedir=f'pretrained/sepformer-wsj02mix',run_opts={"device":"cuda"})
# est_sources = models[model_name].separate_file(path=f'{mix_path}/{file}') 
      
# item_name = file.split("_")[0]

# audio1_filename = f"{output_path}/{item_name}_source1.wav"
# audio2_filename = f"{output_path}/{item_name}_source2.wav"

# torchaudio.save(audio1_filename, est_sources[:, :, 0].detach().cpu(), 8000)
# torchaudio.save(audio2_filename, est_sources[:, :, 1].detach().cpu(), 8000)

Resampling the audio from 48000 Hz to 8000 Hz
